In [1]:
#spark.sql("CREATE TEMPORARY VIEW COPUK_Bronze USING CSV OPTIONS ( path 'Files/Bronze/*/*.csv', header true);")
from pyspark.sql.functions import input_file_name

df = spark.read.option("header", True).csv("Files/Bronze/*/*.csv") \
    .withColumn("source_file", input_file_name())

# Register as temporary view
df.createOrReplaceTempView("COPUK_Bronze")

#display(df)

StatementMeta(, 9c5eb24f-334d-475b-9c9f-06aa0d44bc5e, 3, Finished, Available, Finished)

In [13]:
#display(df)


df = spark.sql("""
    SELECT 
   -- `source_file`,
       SUBSTRING(`source_file`, 143, 7)     AS data_version,
       -- SUBSTRING_INDEX(`source_file`  , "/", 8)  AS data_version,
        --`source_file`                       AS data_version,
        `Title Number`                      AS title_number,
        `Tenure`                            AS tenure,
        `Property Address`                  AS property_address,
        `Region`                            AS region,
        `District`                          AS district,
        `County`                            AS county,
        `Postcode`                          AS postcode,
        `Date Proprietor Added`             AS date_proprietor_added,
        `Price Paid`                        AS price_paid,
        `Company Registration No. (1)`      AS company_registration_1,
        `Additional Proprietor Indicator`   AS additional_proprietor_indicator,
        `Proprietorship Category (1)`       AS proprietorship_category_1,
        `Company Registration No. (2)`      AS company_registration_2,
        `Company Registration No. (3)`      AS company_registration_3,
        `Company Registration No. (4)`      AS company_registration_4, 
        `Proprietor Name (1)`               AS proprietor_name_1,
        `Proprietor Name (2)`               AS proprietor_name_2,
        `Proprietor Name (3)`               AS proprietor_name_3,
        `Proprietor Name (4)`               AS proprietor_name_4,
        `Proprietor (1) Address (1)`        AS proprietor_1_address_1,
        `Proprietor (1) Address (2)`        AS proprietor_1_address_2,
        `Proprietor (1) Address (3)`        AS proprietor_1_address_3,
        `Proprietor (2) Address (1)`        AS proprietor_2_address_1,
        `Proprietor (2) Address (2)`        AS proprietor_2_address_2,
        `Proprietor (2) Address (3)`        AS proprietor_2_address_3,
        `Proprietor (3) Address (1)`        AS proprietor_3_address_1,
        `Proprietor (3) Address (2)`        AS proprietor_3_address_2,
        `Proprietor (3) Address (3)`        AS proprietor_3_address_3,
        `Proprietor (4) Address (1)`        AS proprietor_4_address_1,
        `Proprietor (4) Address (2)`        AS proprietor_4_address_2,
        `Proprietor (4) Address (3)`        AS proprietor_4_address_3,
        `Proprietorship Category (2)`       AS proprietorship_category_2,
        `Proprietorship Category (3)`       AS proprietorship_category_3,
        `Proprietorship Category (4)`       AS proprietorship_category_4,
        `Multiple Address Indicator`        AS multiple_address_indicator
       -- split("source_file", '/')[1]        AS data_version 
   
    FROM COPUK_Bronze
    --WHERE `Price Paid` > 500000
""")
display(df)

StatementMeta(, 9c5eb24f-334d-475b-9c9f-06aa0d44bc5e, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6b902014-e961-4a08-ab7f-31ecb1a016f0)

In [3]:
df.write.mode("overwrite").format("delta").save("Tables/Silver/CCOD_COU_2025_01")

StatementMeta(, bbfeba9a-b7a1-433b-9050-799f95a46355, 5, Finished, Available, Finished)

In [4]:
%%sql
SELECT * FROM silver.CCOD_COU_2025_01

StatementMeta(, bbfeba9a-b7a1-433b-9050-799f95a46355, 6, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 35 fields>